<a href="https://colab.research.google.com/github/mvadrev/coviScan/blob/main/Ensemble_resNet_and_effNet_updated_3_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm https://github.com/ufoym/imbalanced-dataset-sampler/archive/master.zip


     - 297 kB 914 kB/s
     |████████████████████████████████| 431 kB 7.2 MB/s 
  Created wheel for torchsampler: filename=torchsampler-0.1.1-py3-none-any.whl size=3838 sha256=df8c29c358feadf5a334fd52c1f9220f83a83677ccf95727b140f08a3df0b52b
  Stored in directory: /tmp/pip-ephem-wheel-cache-msbvxo3j/wheels/52/7b/7d/ce0e0ddbb7864877a0e31a96f883a928791ebfa6eaf7b52f87
Successfully built torchsampler


In [ ]:
import os 
import shutil
import numpy as np
import glob
import pandas as pd
import seaborn as sns
from collections import Counter
import math

import matplotlib.pyplot as plt
import time
import copy
from random import shuffle
import tqdm.notebook as tqdm


import sklearn
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report
from PIL import Image
import cv2

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.nn import functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data as data
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
from torchsummary import summary
from torchsampler import ImbalancedDatasetSampler
from torch.utils.data.sampler import SubsetRandomSampler, WeightedRandomSampler
from torch.autograd import Variable
import torch.nn.functional as F
# from lime import lime_image
from skimage.segmentation import mark_boundaries
from sklearn import decomposition
from sklearn import manifold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import json
import timm 
import copy
from collections import namedtuple
import os
import random
import shutil
import time

In [ ]:
!unzip /content/data_corr_png.zip

Archive:  /content/data_corr_png.zip
   creating: data/
   creating: data/images/
   creating: data/images/healthy/
  inflating: data/images/healthy/0cough-heavy (2).png  
  inflating: data/images/healthy/0cough-heavy.png  
  inflating: data/images/healthy/0cough-shallow (2).png  
  inflating: data/images/healthy/0cough-shallow.png  
  inflating: data/images/healthy/1000cough-heavy (2).png  
  inflating: data/images/healthy/1000cough-heavy.png  
  inflating: data/images/healthy/1000cough-shallow (2).png  
  inflating: data/images/healthy/1000cough-shallow.png  
  inflating: data/images/healthy/1004cough-heavy (2).png  
  inflating: data/images/healthy/1004cough-heavy.png  
  inflating: data/images/healthy/1004cough-shallow (2).png  
  inflating: data/images/healthy/1004cough-shallow.png  
  inflating: data/images/healthy/1005cough-heavy (2).png  
  inflating: data/images/healthy/1005cough-heavy.png  
  inflating: data/images/healthy/1005cough-shallow (2).png  
  inflating: data/images/

In [ ]:
# Set randonm seed for resproducibility
random.seed(1234)
np.random.seed(1234)
data_dir = "/content/data"
images_dir = os.path.join(data_dir, 'images')
image_files = glob.glob(images_dir+'/*/*')
labels = []
for file_name in image_files:
  labels.append(file_name.split("/")[4])
img_df = pd.DataFrame({'images':image_files, 'labels':labels})
img_df.head()


,images,labels
0,/content/data/images/positive/19061906cough-he...,positive
1,/content/data/images/positive/18911891cough-sh...,positive
2,/content/data/images/positive/12461246cough-sh...,positive
3,/content/data/images/positive/19071907cough-he...,positive
4,/content/data/images/positive/1857cough-shallo...,positive


In [ ]:
# Split dataset into train , test and validation
train, test = train_test_split(img_df, stratify=img_df.labels, test_size=0.1, random_state=42)
train, valid = train_test_split(train, stratify=train.labels, test_size=0.1, random_state=42)

In [ ]:
#creating train, test and validation directories
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
val_dir = os.path.join(data_dir, 'val')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir) 
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
if os.path.exists(val_dir):
    shutil.rmtree(val_dir)

os.makedirs(train_dir)
os.makedirs(test_dir)
os.makedirs(val_dir)

classes = os.listdir(images_dir)
print("There are",classes,"in the data")

for c in classes:   
    train_filter = train[train.labels==c]
    test_filter = test[test.labels==c]
    val_filter = valid[valid.labels==c]

    train_filter_images = train_filter.images.tolist()
    test_filter_images = test_filter.images.tolist()
    val_filter_images = val_filter.images.tolist()

    os.makedirs(os.path.join(train_dir, c), exist_ok = True)
    os.makedirs(os.path.join(test_dir, c), exist_ok = True)
    os.makedirs(os.path.join(val_dir, c), exist_ok = True)

    for image in train_filter_images:

        image_name = image.split("/")[5]
        image_dst = os.path.join(train_dir, c, image_name) 
        shutil.copyfile(image, image_dst)
        
    for image in test_filter_images:
        image_name = image.split("/")[5]
        image_dst = os.path.join(test_dir, c, image_name) 
        shutil.copyfile(image, image_dst)

    for image in val_filter_images:
        image_name = image.split("/")[5]
        image_dst = os.path.join(val_dir, c, image_name) 
        shutil.copyfile(image, image_dst)

There are ['positive', 'healthy'] in the data


In [ ]:
import timm 
import torch
import torch.nn as nn

# Load resNet50
model_name = 'resnet50'
resnet50_model = timm.create_model('resnet50',pretrained=True) #load pretrained model

for param in resnet50_model.parameters():
  param.requires_grad = False

# Change the final layer of ResNet50 Model for Transfer Learning
fc_inputs = resnet50_model.fc.in_features
resnet50_model.fc = nn.Sequential(  nn.Linear(in_features=fc_inputs, out_features=625),
                              nn.ReLU(), #ReLu to be the activation function
                              nn.Dropout(p=0.3),
                              nn.Linear(in_features=625, out_features=256),
                              nn.ReLU(),
                              nn.Linear(in_features=256, out_features=2), 
                              )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50_model.to(device) # move the model to GPU
resnet50_model.load_state_dict(torch.load('/content/resnet.pt'))
resnet50_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act3): ReLU(inplace=True)
      (downsample): Sequen

In [ ]:
def show_image(image,label,get_denormalize = True):
    
    image = image.permute(1,2,0)
    mean = torch.FloatTensor([0.485, 0.456, 0.406])
    std = torch.FloatTensor([0.229, 0.224, 0.225])
    
    if get_denormalize == True:
        image = image*std + mean
        image = np.clip(image,0,1)
        plt.imshow(image)
        plt.title(label)
        
    else: 
        plt.imshow(image)
        plt.title(label)

def show_grid(image,title = None):
    
    image = image.permute(1,2,0)
    mean = torch.FloatTensor([0.485, 0.456, 0.406])
    std = torch.FloatTensor([0.229, 0.224, 0.225])
    
    image = image*std + mean
    image = np.clip(image,0,1)
    
    plt.figure(figsize=[15, 15])
    plt.imshow(image)
    if title != None:
        plt.title(title)


def accuracy(y_pred,y_true):
    y_pred = F.softmax(y_pred,dim = 1)
    top_p,top_class = y_pred.topk(1,dim = 1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))


def view_classify(image,ps,label):
    
    class_name = ['NORMAL', 'POSITIVE']
    classes = np.array(class_name)

    ps = ps.cpu().data.numpy().squeeze()
    
    image = image.permute(1,2,0)
    mean = torch.FloatTensor([0.485, 0.456, 0.406])
    std = torch.FloatTensor([0.229, 0.224, 0.225])
    
    
    image = image*std + mean
    img = np.clip(image,0,1)
    
    fig, (ax1, ax2) = plt.subplots(figsize=(8,12), ncols=2)
    ax1.imshow(img)
    ax1.set_title('Ground Truth : {}'.format(class_name[label]))
    ax1.axis('off')
    ax2.barh(classes, ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(classes)
    ax2.set_yticklabels(classes)
    ax2.set_title('Predicted Class')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

    return None

In [ ]:
class COVIDTrainer():
    
    def __init__(self,criterion = None,optimizer = None,schedular = None):
        
        self.criterion = criterion
        self.optimizer = optimizer
        self.schedular = schedular
        self.history = []
    
    def train_batch_loop(self,model,trainloader):
        
        train_loss = 0.0
        train_acc = 0.0
        
        for images,labels in tqdm.tqdm(trainloader): 
            
            # move the data to CPU
            images = images.to(device)
            labels = labels.to(device)
            
            logits = model(images)
            loss = self.criterion(logits,labels)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            train_loss += loss.item()
            train_acc += accuracy(logits,labels)
            
        return train_loss / len(trainloader), train_acc / len(trainloader) 

    
    def valid_batch_loop(self,model,validationloader):
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for images,labels in tqdm.tqdm(validationloader):
            
            # move the data to CPU
            images = images.to(device) 
            labels = labels.to(device)
            
            logits = model(images)
            loss = self.criterion(logits,labels)
            
            valid_loss += loss.item()
            valid_acc += accuracy(logits,labels)
            
        return valid_loss / len(validationloader), valid_acc / len(validationloader)
            
        
    def fit(self,model,trainloader,validationloader,epochs):
        
        valid_min_loss = np.Inf 
        
        for i in range(epochs):
            
            model.train() # this turn on dropout
            avg_train_loss, avg_train_acc = self.train_batch_loop(model,trainloader) ###
            
            model.eval()  # this turns off the dropout lapyer and batch norm
            avg_valid_loss, avg_valid_acc = self.valid_batch_loop(model,validationloader) ###
            
            if avg_valid_loss <= valid_min_loss :
                print("Valid_loss decreased {} --> {}".format(valid_min_loss,avg_valid_loss))
                torch.save(model.state_dict(),'ColabCovidModel.pt')
                valid_min_loss = avg_valid_loss

                
            print("Epoch : {} Train Loss : {:.6f} Train Acc : {:.6f}".format(i+1, avg_train_loss, avg_train_acc))
            print("Epoch : {} Valid Loss : {:.6f} Valid Acc : {:.6f}".format(i+1, avg_valid_loss, avg_valid_acc))
            self.history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
        return self.history

In [ ]:
class CFG:

  epochs =25                          
  lr = 0.001                           
  batch_size = 16                      
  img_size = 224                       

  # going to be used for loading dataset
  train_path='/content/data/train'
  validate_path='/content/data/val'
  test_path='/content/data/test'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("On which device we are on:{}".format(device))

On which device we are on:cuda


In [ ]:
!unzip data_corr_png.zip

In [ ]:
#transforming images
train_transform = transforms.Compose([  
                             transforms.Resize(size=(CFG.img_size,CFG.img_size)), # Resizing the image to be 224 by 224
                             transforms.RandomHorizontalFlip(),
                             transforms.RandomVerticalFlip(),
                             transforms.RandomRotation(degrees=(-10,+10)), 
                             transforms.ToTensor(),  
                             transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]) 

])

validate_transform = transforms.Compose([  
                             transforms.Resize(size=(CFG.img_size,CFG.img_size)), 
                             transforms.ToTensor(), 
                             transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]) 
])

test_transform = transforms.Compose([ 
                             transforms.Resize(size=(CFG.img_size,CFG.img_size)), 
                             transforms.ToTensor(),
                             transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]) 

])


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet50_model.parameters(),lr = CFG.lr)
trainer = COVIDTrainer(criterion,optimizer)

In [ ]:
trainset=datasets.ImageFolder(CFG.train_path,transform=train_transform)
print("Trainset Size:  {}".format(len(trainset)))

validateset=datasets.ImageFolder(CFG.validate_path,transform=validate_transform)
print("validateset Size:  {}".format(len(validateset)))

testset=datasets.ImageFolder(CFG.test_path,transform=test_transform)
print("testset Size:  {}".format(len(testset)))

Trainset Size:  2572
validateset Size:  286
testset Size:  318


In [ ]:
trainloader = data.DataLoader(trainset,batch_size=CFG.batch_size,shuffle=True)
print("No. of batches in trainloader:{}".format(len(trainloader))) #Trainset Size:  3101 / batch_size: 16 = 194(No. of batches in trainloader) 
print("No. of Total examples:{}".format(len(trainloader.dataset)))

validationloader = data.DataLoader(validateset,batch_size=CFG.batch_size,shuffle=True)
print("No. of batches in validationloader:{}".format(len(validationloader))) #validationset Size:  345 / batch_size: 16 = 22(No. of batches in validationloader) 
print("No. of Total examples:{}".format(len(validationloader.dataset)))

testloader = data.DataLoader(testset,batch_size=CFG.batch_size,shuffle=True)
print("No. of batches in testloader:{}".format(len(testloader))) #testset Size:  383 / batch_size: 16 = 24(No. of batches in testloader) 
print("No. of Total examples:{}".format(len(testloader.dataset)))

No. of batches in trainloader:161
No. of Total examples:2572
No. of batches in validationloader:18
No. of Total examples:286
No. of batches in testloader:20
No. of Total examples:318


In [ ]:
def computeTestSetAccuracy(model, criterion):
    '''
    Function to compute the accuracy on the test set
    Parameters
        :param model: Model to test
        :param loss_criterion: Loss Criterion to minimize
    '''

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    test_acc = 0.0
    test_loss = 0.0

    # Set to evaluation mode
    model.eval()

    # Validation loop
    y_pred_list = []
    y_true_list = []
    for j, (images, labels) in enumerate(testloader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass - compute outputs on input data using the model
        logits = model(images)

        y_true_list.append(labels.data.to("cpu").tolist())
        #y_pred_list.append(logits.to("cpu").tolist())
        y_pred = F.softmax(logits,dim = 1)
        top_p,top_class = y_pred.topk(1,dim = 1)
        y_pred_list.append(top_class.data.to("cpu").tolist())

        loss = criterion(logits,labels)
        test_loss += loss.item()
        test_acc += accuracy(logits,labels)

    return y_pred_list, y_true_list


In [ ]:
avg_test_loss_resNet, avg_test_acc_resNet = trainer.valid_batch_loop(resnet50_model,testloader)
print("Test Loss : {}".format(avg_test_loss_resNet))
print("Test Acc : {}".format(avg_test_acc_resNet))
y_pred_list, y_true_list = computeTestSetAccuracy(resnet50_model, criterion)
y_pred_list_resNet = [j for i in y_pred_list for j in i]
y_true_list_resNet = [j for i in y_true_list for j in i]
print(classification_report(y_true_list_resNet, y_pred_list_resNet, digits=3))

  0%|          | 0/20 [00:00<?, ?it/s]

Test Loss : 0.6481381714344024
Test Acc : 0.6540178656578064
              precision    recall  f1-score   support

           0      0.679     0.855     0.757       200
           1      0.561     0.314     0.402       118

    accuracy                          0.654       318
   macro avg      0.620     0.584     0.579       318
weighted avg      0.635     0.654     0.625       318



In [ ]:
# Load effNet saved model
model_name = 'efficientnetv2_rw_m'
import tqdm.notebook as tqdm

efficientnetv2_model = timm.create_model(model_name,pretrained=True) #load pretrained model
# Freeze model parameters
for param in efficientnetv2_model.parameters():
  param.requires_grad = False

# Change the final layer of Efficientnetv2 Model for Transfer Learning
#we are updating it as a 2-class classifier:
efficientnetv2_model.classifier = nn.Sequential(
    nn.Linear(in_features=2152, out_features=625), #2152 is the orginal in_features
    nn.ReLU(), #ReLu to be the activation function
    nn.Dropout(p=0.3),
    nn.Linear(in_features=625, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=2), 
)

# as we are training on GPU. Convert model to be used on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
efficientnetv2_model.to(device) # move the model to GPU
efficientnetv2_model.load_state_dict(torch.load('/content/effnet.pt'))
efficientnetv2_model.eval()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnetv2_rw_m_agc-3d90cb1e.pth" to /root/.cache/torch/hub/checkpoints/efficientnetv2_rw_m_agc-3d90cb1e.pth


EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): EdgeResidual(
        (conv_exp): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): Identity()
        (conv_pwl): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): EdgeResidual(
        (conv_exp): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): I

In [ ]:
criterion2 = nn.CrossEntropyLoss()
optimizer2 = torch.optim.Adam(efficientnetv2_model.parameters(),lr = CFG.lr)
trainer2 = COVIDTrainer(criterion2,optimizer2)

In [ ]:
avg_test_loss_effNet, avg_test_acc_effNet = trainer2.valid_batch_loop(efficientnetv2_model,testloader)


print("Test Loss : {}".format(avg_test_loss_effNet))
print("Test Acc : {}".format(avg_test_acc_effNet))

import sklearn
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report

y_pred_list_effNet, y_true_list_effNet = computeTestSetAccuracy(efficientnetv2_model, criterion)

y_pred_list_effNet = [j for i in y_pred_list for j in i]
y_true_list_effNet = [j for i in y_true_list for j in i]

print(classification_report(y_true_list_effNet, y_pred_list_effNet, digits=3))

  0%|          | 0/20 [00:00<?, ?it/s]

Test Loss : 0.5680970251560211
Test Acc : 0.7169643044471741
              precision    recall  f1-score   support

           0      0.679     0.855     0.757       200
           1      0.561     0.314     0.402       118

    accuracy                          0.654       318
   macro avg      0.620     0.584     0.579       318
weighted avg      0.635     0.654     0.625       318



In [ ]:
# implement ensemble 
# Inputs all images in test loader p
from sklearn.ensemble import VotingClassifier

resNetResults = list(zip(y_pred_list_resNet, y_true_list_resNet))
effNetResults = zip(y_pred_list_effNet, y_true_list_effNet)
print("ResNet", resNetResults)

ResNet [([0], 0), ([0], 1), ([0], 0), ([0], 1), ([1], 1), ([1], 1), ([0], 1), ([0], 0), ([1], 1), ([0], 0), ([0], 1), ([0], 0), ([0], 0), ([0], 0), ([0], 0), ([0], 0), ([1], 1), ([0], 0), ([1], 0), ([0], 0), ([0], 1), ([0], 0), ([1], 1), ([1], 0), ([0], 0), ([0], 0), ([0], 0), ([0], 0), ([0], 0), ([1], 1), ([0], 0), ([0], 1), ([0], 0), ([0], 0), ([0], 1), ([0], 0), ([0], 0), ([0], 0), ([0], 0), ([1], 1), ([0], 0), ([0], 0), ([1], 0), ([0], 0), ([0], 1), ([0], 0), ([1], 0), ([1], 1), ([0], 0), ([0], 1), ([0], 0), ([0], 0), ([0], 0), ([1], 1), ([0], 0), ([1], 1), ([0], 1), ([0], 0), ([0], 0), ([0], 0), ([0], 0), ([0], 0), ([0], 0), ([1], 1), ([0], 0), ([0], 0), ([0], 0), ([0], 1), ([0], 0), ([0], 0), ([0], 0), ([0], 0), ([0], 1), ([1], 0), ([0], 0), ([0], 1), ([0], 1), ([1], 1), ([1], 1), ([1], 1), ([0], 0), ([1], 1), ([1], 0), ([0], 0), ([0], 1), ([0], 1), ([1], 0), ([0], 1), ([0], 1), ([0], 0), ([1], 1), ([0], 0), ([0], 1), ([0], 1), ([1], 0), ([0], 0), ([0], 0), ([1], 0), ([0], 0), ([

In [ ]:
resNetResults[0]

([0], 0)

In [ ]:
votingclass = VotingClassifier(estimators= [('resnet50' , resnet50_model ), ('effNetv2' , efficientnetv2_model)], voting='soft')
votingclass

VotingClassifier(estimators=[('resnet50',
                              ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv...
  (act2): SiLU(inplace=True)
  (global_pool): SelectAdaptivePool2d (pool_type=avg, flatten=Flatten(start_dim=1, end_dim=-1))
  (classifier): Sequential(
    (0): Linear(in_features=2152, out_features=625, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=625, out_features=256, bias=True)
    (4): ReLU()
    (5): Linear(in_features=256, out_features=2, bias=True)
  )
))],
                 voting='soft')

In [ ]:
history = trainer.fit(votingclass,testloader,testloader,epochs = 24)


AttributeError: ignored

In [ ]:
!pip install torchensemble

     |████████████████████████████████| 40 kB 4.8 MB/s 


In [ ]:
from torchensemble.utils.logging import set_logger

logger = set_logger('classification_mnist_mlp')

Log will be saved in '/content/logs'.
Create folder 'logs/'
Start logging into file /content/logs/classification_mnist_mlp-2022_03_29_17_25.log...


In [ ]:
from torchensemble import VotingClassifier

model = VotingClassifier(
    estimator=resnet50_model,
    n_estimators=5,
    cuda=True,
)

In [ ]:
criterion = nn.CrossEntropyLoss()
model.set_criterion(criterion)

In [ ]:
model.set_optimizer('Adam',             # parameter optimizer
                    lr=1e-3,            # learning rate of the optimizer
                    weight_decay=5e-4)  # weight decay of the optimizer

In [ ]:
from PIL import Image

sample = Image.open("/content/data/test/healthy/1005cough-shallow (2).png")

In [ ]:
accuracy = model.predict(torch.tensor(sample))

RuntimeError: ignored

In [ ]:
# Training
model.fit(train_loader=trainloader,  # training data
          epochs=30)                 # the number of training epochs

Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 0.51984 | Correct: 12/16
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 0.61622 | Correct: 10/16
Estimator: 001 | Epoch: 000 | Batch: 000 | Loss: 0.72993 | Correct: 9/16
Estimator: 001 | Epoch: 000 | Batch: 100 | Loss: 0.48887 | Correct: 12/16
Estimator: 002 | Epoch: 000 | Batch: 000 | Loss: 0.57306 | Correct: 11/16
Estimator: 002 | Epoch: 000 | Batch: 100 | Loss: 0.65378 | Correct: 9/16
Estimator: 003 | Epoch: 000 | Batch: 000 | Loss: 0.60530 | Correct: 9/16
Estimator: 003 | Epoch: 000 | Batch: 100 | Loss: 0.60827 | Correct: 10/16
Estimator: 004 | Epoch: 000 | Batch: 000 | Loss: 0.70166 | Correct: 9/16
Estimator: 004 | Epoch: 000 | Batch: 100 | Loss: 0.65278 | Correct: 11/16
Estimator: 000 | Epoch: 001 | Batch: 000 | Loss: 0.57091 | Correct: 12/16
Estimator: 000 | Epoch: 001 | Batch: 100 | Loss: 0.59396 | Correct: 11/16
Estimator: 001 | Epoch: 001 | Batch: 000 | Loss: 0.65541 | Correct: 11/16
Estimator: 001 | Epoch: 001 | Batch: 100 |

2022-03-29 18:07:19,650 - INFO: Saving the model to `./VotingClassifier_ResNet_5_ckpt.pth`


In [ ]:
accuracy = model.evaluate(testloader)

In [ ]:
idx, label = next(iter(testloader))

In [ ]:
label

tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [ ]:
idx.shape

torch.Size([16, 3, 224, 224])

In [ ]:
output = model.predict(idx)

In [ ]:
 _, predicted = torch.max(output.data, 1)

In [ ]:
accuracy

71.38364779874213

In [ ]:
predicted

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0])

In [ ]:
label

tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [ ]:
a = label.tolist()
a

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0]

In [ ]:

print(classification_report(label.tolist(), predicted.tolist(), digits=3))

              precision    recall  f1-score   support

           0      0.769     0.909     0.833        11
           1      0.667     0.400     0.500         5

    accuracy                          0.750        16
   macro avg      0.718     0.655     0.667        16
weighted avg      0.737     0.750     0.729        16



In [77]:
from torchensemble import VotingClassifier
# effNet Ensemble
model2 = VotingClassifier(
    estimator=resnet50_model,
    n_estimators=2,
    cuda=True,
)

In [78]:
model2.set_optimizer('Adam',             # parameter optimizer
                    lr=1e-3,            # learning rate of the optimizer
                    weight_decay=5e-4)  # weight decay of the optimizer

In [82]:
# Training
model2.fit(train_loader=trainloader,  # training data
          epochs=30)                 # the number of training epochs

Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 0.51207 | Correct: 12/16
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 0.72915 | Correct: 10/16
Estimator: 001 | Epoch: 000 | Batch: 000 | Loss: 1.03620 | Correct: 8/16
Estimator: 001 | Epoch: 000 | Batch: 100 | Loss: 0.49359 | Correct: 13/16
Estimator: 000 | Epoch: 001 | Batch: 000 | Loss: 0.57463 | Correct: 9/16
Estimator: 000 | Epoch: 001 | Batch: 100 | Loss: 0.47583 | Correct: 13/16
Estimator: 001 | Epoch: 001 | Batch: 000 | Loss: 0.61379 | Correct: 11/16
Estimator: 001 | Epoch: 001 | Batch: 100 | Loss: 0.61917 | Correct: 10/16
Estimator: 000 | Epoch: 002 | Batch: 000 | Loss: 0.61028 | Correct: 12/16
Estimator: 000 | Epoch: 002 | Batch: 100 | Loss: 0.62820 | Correct: 11/16
Estimator: 001 | Epoch: 002 | Batch: 000 | Loss: 0.56723 | Correct: 10/16
Estimator: 001 | Epoch: 002 | Batch: 100 | Loss: 0.61601 | Correct: 9/16
Estimator: 000 | Epoch: 003 | Batch: 000 | Loss: 0.39168 | Correct: 15/16
Estimator: 000 | Epoch: 003 | Batch: 100 

2022-03-29 19:03:46,614 - INFO: Saving the model to `./VotingClassifier_ResNet_2_ckpt.pth`


In [83]:
model2.evaluate(testloader)

77.9874213836478

In [85]:
output = model2.predict(idx)

In [86]:
 _, predicted2 = torch.max(output.data, 1)

In [93]:
label

tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [94]:
predicted2

tensor([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [92]:
print(classification_report(label.tolist(), predicted2.tolist(), digits=3))

              precision    recall  f1-score   support

           0      0.727     0.727     0.727        11
           1      0.400     0.400     0.400         5

    accuracy                          0.625        16
   macro avg      0.564     0.564     0.564        16
weighted avg      0.625     0.625     0.625        16

